In [1]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime
#载入数据
mnist = input_data.read_data_sets("/root/code/MNIST_data/",one_hot=True)
#载入图片是 28*28
n_inputs = 28 #输入一共28行
max_time = 28 #一共28行
lstm_size = 100#隐层单元 block
n_classes = 10 #10个分类
batch_size = 50 #每批次50个样本
n_batch = mnist.train.num_examples//batch_size #计算一共有多少批次

#这里的none 表示第一维度可以为任意长度
x = tf.placeholder(tf.float32,[None,784],name='x')
#正确的标签
y = tf.placeholder(tf.float32,[None,10],name='y')

#初始化权重
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
#初始化偏置
biases =tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义run网络
def RUN(X,weights,biases,re_name):
    # inputs = [batch_size,max_time,n_inputs]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    """
    tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True): 
    n_hidden表示神经元的个数，
    forget_bias就是LSTM们的忘记系数，默认为1，如果等于1，就是不会忘记任何信息。如果等于0，就都忘记。
    state_is_tuple默认就是True，官方建议用True，就是表示返回的状态用一个元祖表示。
    这个里面存在一个状态初始化函数，就是zero_state（batch_size，dtype）两个参数。
    batch_size就是输入样本批次的数目，
    dtype就是数据类型。
    """
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    #final_state[0]是cell state
    #final_state[1]是hidden state
    """
    tf.nn.dynamic_rnn(cell,inputs,sequence_length=None,initial_state=None,dtype=None,parallel_iterations=None,swap_memory=False,
    time_major=False,scope=None )
    cell: RNNCell的一个实例.

inputs: RNN输入.

如果time_major == False(默认), 则是一个shape为[batch_size, max_time, input_size]的Tensor,或者这些元素的嵌套元组。

如果time_major == True,则是一个shape为[max_time, batch_size, input_size]的Tensor,或这些元素的嵌套元组。

sequence_length: (可选）大小为[batch_size],数据的类型是int32/int64向量。如果当前时间步的index超过该序列的实际长度时，
则该时间步不进行计算，RNN的state复制上一个时间步的，同时该时间步的输出全部为零。

initial_state: (可选)RNN的初始state(状态)。如果cell.state_size(一层的RNNCell)是一个整数，
那么它必须是一个具有适当类型和形状的张量[batch_size，cell.state_size]。如果cell.state_size是一个元组(多层的RNNCell,如MultiRNNCell)，
那么它应该是一个张量元组，每个元素的形状为[batch_size，s] for s in cell.state_size。

time_major: inputs 和outputs 张量的形状格式。如果为True，则这些张量都应该是（都会是）[max_time, batch_size, depth]。
如果为false，则这些张量都应该是（都会是）[batch_size，max_time, depth]。time_major=true说明输入和输出tensor的第一维是max_time。
否则为batch_size。

使用time_major =True更有效,因为它避免了RNN计算开始和结束时的转置.但是,大多数TensorFlow数据都是batch-major,因此默认情况下,
此函数接受输入并以batch-major形式发出输出.

返回值：

一对(outputs, state),其中：

outputs： RNN输出Tensor.

如果time_major == False(默认),这将是shape为[batch_size, max_time, cell.output_size]的Tensor.

如果time_major == True,这将是shape为[max_time, batch_size, cell.output_size]的Tensor.

state： 最终的状态.

一般情况下state的形状为 [batch_size, cell.output_size ]

如果cell是LSTMCells,则state将是包含每个单元格的LSTMStateTuple的元组，state的形状为[2，batch_size, cell.output_size ]

    """
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases,name=re_name)
    return results

#计算RUN的返回结果
result = "result"
prediction = RUN(x,weights,biases,result)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels =y))
#使用Adamoptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

saver = tf.train.Saver() #定义saver


#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start = datetime.now()
    for epoch in range(5001):
#         for batch in range(n_batch):
        batch_xs,batch_ys =mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter"+str(epoch)+",testing accuracy="+ str(acc))
    end = datetime.now()
    latency = (end - start).total_seconds()
    print('训练5000步总时长为 '+str(latency)+' 秒 ')
    saver.save(sess, '/tmp/tf_rnn_1/model.ckpt') #模型储存位置




/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /root/code/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /root/code/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Pleas

Iter141,testing accuracy=0.2345
Iter142,testing accuracy=0.2365
Iter143,testing accuracy=0.2383
Iter144,testing accuracy=0.239
Iter145,testing accuracy=0.2399
Iter146,testing accuracy=0.2414
Iter147,testing accuracy=0.2432
Iter148,testing accuracy=0.2455
Iter149,testing accuracy=0.2463
Iter150,testing accuracy=0.2482
Iter151,testing accuracy=0.2504
Iter152,testing accuracy=0.2515
Iter153,testing accuracy=0.2522
Iter154,testing accuracy=0.2522
Iter155,testing accuracy=0.2529
Iter156,testing accuracy=0.2529
Iter157,testing accuracy=0.2532
Iter158,testing accuracy=0.2535
Iter159,testing accuracy=0.2528
Iter160,testing accuracy=0.2529
Iter161,testing accuracy=0.2532
Iter162,testing accuracy=0.2551
Iter163,testing accuracy=0.2579
Iter164,testing accuracy=0.2602
Iter165,testing accuracy=0.263
Iter166,testing accuracy=0.2653
Iter167,testing accuracy=0.2676
Iter168,testing accuracy=0.27
Iter169,testing accuracy=0.2737
Iter170,testing accuracy=0.277
Iter171,testing accuracy=0.2799
Iter172,testi

Iter398,testing accuracy=0.4698
Iter399,testing accuracy=0.4697
Iter400,testing accuracy=0.4679
Iter401,testing accuracy=0.4679
Iter402,testing accuracy=0.4672
Iter403,testing accuracy=0.4659
Iter404,testing accuracy=0.465
Iter405,testing accuracy=0.4656
Iter406,testing accuracy=0.4683
Iter407,testing accuracy=0.4704
Iter408,testing accuracy=0.4716
Iter409,testing accuracy=0.4727
Iter410,testing accuracy=0.4747
Iter411,testing accuracy=0.4761
Iter412,testing accuracy=0.4781
Iter413,testing accuracy=0.4797
Iter414,testing accuracy=0.48
Iter415,testing accuracy=0.4798
Iter416,testing accuracy=0.4803
Iter417,testing accuracy=0.4822
Iter418,testing accuracy=0.4818
Iter419,testing accuracy=0.4836
Iter420,testing accuracy=0.4838
Iter421,testing accuracy=0.4849
Iter422,testing accuracy=0.4854
Iter423,testing accuracy=0.4874
Iter424,testing accuracy=0.4889
Iter425,testing accuracy=0.4894
Iter426,testing accuracy=0.4932
Iter427,testing accuracy=0.4967
Iter428,testing accuracy=0.498
Iter429,test

Iter656,testing accuracy=0.6267
Iter657,testing accuracy=0.627
Iter658,testing accuracy=0.627
Iter659,testing accuracy=0.6258
Iter660,testing accuracy=0.6244
Iter661,testing accuracy=0.6228
Iter662,testing accuracy=0.6232
Iter663,testing accuracy=0.6217
Iter664,testing accuracy=0.6203
Iter665,testing accuracy=0.6183
Iter666,testing accuracy=0.6172
Iter667,testing accuracy=0.617
Iter668,testing accuracy=0.6173
Iter669,testing accuracy=0.6156
Iter670,testing accuracy=0.6185
Iter671,testing accuracy=0.6194
Iter672,testing accuracy=0.622
Iter673,testing accuracy=0.6226
Iter674,testing accuracy=0.6226
Iter675,testing accuracy=0.6247
Iter676,testing accuracy=0.6249
Iter677,testing accuracy=0.6258
Iter678,testing accuracy=0.6276
Iter679,testing accuracy=0.6265
Iter680,testing accuracy=0.6262
Iter681,testing accuracy=0.6239
Iter682,testing accuracy=0.619
Iter683,testing accuracy=0.6159
Iter684,testing accuracy=0.6138
Iter685,testing accuracy=0.617
Iter686,testing accuracy=0.6213
Iter687,testin

...

Iter3914,testing accuracy=0.8877
Iter3915,testing accuracy=0.8866
Iter3916,testing accuracy=0.8852
Iter3917,testing accuracy=0.8842
Iter3918,testing accuracy=0.8845
Iter3919,testing accuracy=0.8843
Iter3920,testing accuracy=0.8839
Iter3921,testing accuracy=0.8846
Iter3922,testing accuracy=0.8862
Iter3923,testing accuracy=0.8887
Iter3924,testing accuracy=0.8898
Iter3925,testing accuracy=0.8902
Iter3926,testing accuracy=0.8896
Iter3927,testing accuracy=0.8896
Iter3928,testing accuracy=0.8891
Iter3929,testing accuracy=0.8887
Iter3930,testing accuracy=0.887
Iter3931,testing accuracy=0.8852
Iter3932,testing accuracy=0.8827
Iter3933,testing accuracy=0.8813
Iter3934,testing accuracy=0.8801
Iter3935,testing accuracy=0.8781
Iter3936,testing accuracy=0.8796
Iter3937,testing accuracy=0.8828
Iter3938,testing accuracy=0.8848
Iter3939,testing accuracy=0.8872
Iter3940,testing accuracy=0.8887
Iter3941,testing accuracy=0.8899
Iter3942,testing accuracy=0.889
Iter3943,testing accuracy=0.8881
Iter3944,tes

Iter4164,testing accuracy=0.8938
Iter4165,testing accuracy=0.8947
Iter4166,testing accuracy=0.8953
Iter4167,testing accuracy=0.8942
Iter4168,testing accuracy=0.8933
Iter4169,testing accuracy=0.8926
Iter4170,testing accuracy=0.8922
Iter4171,testing accuracy=0.8934
Iter4172,testing accuracy=0.892
Iter4173,testing accuracy=0.8922
Iter4174,testing accuracy=0.8926
Iter4175,testing accuracy=0.8944
Iter4176,testing accuracy=0.8934
Iter4177,testing accuracy=0.8917
Iter4178,testing accuracy=0.8922
Iter4179,testing accuracy=0.8923
Iter4180,testing accuracy=0.8931
Iter4181,testing accuracy=0.8942
Iter4182,testing accuracy=0.8949
Iter4183,testing accuracy=0.8958
Iter4184,testing accuracy=0.8962
Iter4185,testing accuracy=0.8955
Iter4186,testing accuracy=0.8964
Iter4187,testing accuracy=0.8951
Iter4188,testing accuracy=0.8938
Iter4189,testing accuracy=0.893
Iter4190,testing accuracy=0.8923
Iter4191,testing accuracy=0.8935
Iter4192,testing accuracy=0.894
Iter4193,testing accuracy=0.8955
Iter4194,test

Iter4414,testing accuracy=0.9017
Iter4415,testing accuracy=0.9007
Iter4416,testing accuracy=0.9022
Iter4417,testing accuracy=0.9008
Iter4418,testing accuracy=0.9006
Iter4419,testing accuracy=0.9002
Iter4420,testing accuracy=0.9008
Iter4421,testing accuracy=0.9022
Iter4422,testing accuracy=0.9016
Iter4423,testing accuracy=0.9018
Iter4424,testing accuracy=0.9013
Iter4425,testing accuracy=0.9009
Iter4426,testing accuracy=0.9002
Iter4427,testing accuracy=0.9005
Iter4428,testing accuracy=0.9
Iter4429,testing accuracy=0.8989
Iter4430,testing accuracy=0.9001
Iter4431,testing accuracy=0.9025
Iter4432,testing accuracy=0.9032
Iter4433,testing accuracy=0.9037
Iter4434,testing accuracy=0.9042
Iter4435,testing accuracy=0.9045
Iter4436,testing accuracy=0.905
Iter4437,testing accuracy=0.9054
Iter4438,testing accuracy=0.9052
Iter4439,testing accuracy=0.9042
Iter4440,testing accuracy=0.9022
Iter4441,testing accuracy=0.9009
Iter4442,testing accuracy=0.8992
Iter4443,testing accuracy=0.8981
Iter4444,testi

Iter4664,testing accuracy=0.9041
Iter4665,testing accuracy=0.9037
Iter4666,testing accuracy=0.9045
Iter4667,testing accuracy=0.9055
Iter4668,testing accuracy=0.907
Iter4669,testing accuracy=0.9058
Iter4670,testing accuracy=0.905
Iter4671,testing accuracy=0.9047
Iter4672,testing accuracy=0.9044
Iter4673,testing accuracy=0.9037
Iter4674,testing accuracy=0.9056
Iter4675,testing accuracy=0.9053
Iter4676,testing accuracy=0.9049
Iter4677,testing accuracy=0.9046
Iter4678,testing accuracy=0.906
Iter4679,testing accuracy=0.9082
Iter4680,testing accuracy=0.9072
Iter4681,testing accuracy=0.9055
Iter4682,testing accuracy=0.9047
Iter4683,testing accuracy=0.9035
Iter4684,testing accuracy=0.9023
Iter4685,testing accuracy=0.9013
Iter4686,testing accuracy=0.9012
Iter4687,testing accuracy=0.901
Iter4688,testing accuracy=0.8997
Iter4689,testing accuracy=0.8991
Iter4690,testing accuracy=0.8987
Iter4691,testing accuracy=0.8984
Iter4692,testing accuracy=0.8986
Iter4693,testing accuracy=0.9001
Iter4694,testi

Iter4914,testing accuracy=0.9112
Iter4915,testing accuracy=0.9098
Iter4916,testing accuracy=0.91
Iter4917,testing accuracy=0.9068
Iter4918,testing accuracy=0.9055
Iter4919,testing accuracy=0.906
Iter4920,testing accuracy=0.9066
Iter4921,testing accuracy=0.9064
Iter4922,testing accuracy=0.9066
Iter4923,testing accuracy=0.9056
Iter4924,testing accuracy=0.9055
Iter4925,testing accuracy=0.9082
Iter4926,testing accuracy=0.9098
Iter4927,testing accuracy=0.911
Iter4928,testing accuracy=0.9122
Iter4929,testing accuracy=0.913
Iter4930,testing accuracy=0.9141
Iter4931,testing accuracy=0.9141
Iter4932,testing accuracy=0.9146
Iter4933,testing accuracy=0.9152
Iter4934,testing accuracy=0.9147
Iter4935,testing accuracy=0.9143
Iter4936,testing accuracy=0.9127
Iter4937,testing accuracy=0.9111
Iter4938,testing accuracy=0.9103
Iter4939,testing accuracy=0.9088
Iter4940,testing accuracy=0.9082
Iter4941,testing accuracy=0.9078
Iter4942,testing accuracy=0.9091
Iter4943,testing accuracy=0.9097
Iter4944,testin

此处说明for循环嵌套 sess训练了 更多的次数 可以去除 训练效果是一样的